# 한글-Claude-v2 Model: Conversational Interface - Chatbot with Claude LLM

> 이 노트북은 SageMaker Studio의 **`Data Science 3.0`** 커널과 잘 작동합니다.

> **SageMaker Notebook Instance** 를 이용해 실습을 진행하신다면, **JupyterLab이 아닌 Jupyter**에서 실행하시기 바랍니다.

이 노트북에서는 Amazon Bedrock의 기본 모델 (FM) 을 사용하여 챗봇을 구축할 것입니다.사용 사례에서는 Claude를 챗봇 구축을 위한 FM으로 사용합니다.

---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

## 개요

챗봇 및 가상 어시스턴트와 같은 대화형 인터페이스를 사용하여 고객의 사용자 경험을 향상시킬 수 있습니다. 챗봇은 자연어 처리 (NLP) 및 기계 학습 알고리즘을 사용하여 사용자 쿼리를 이해하고 이에 응답합니다.챗봇은 고객 서비스, 판매, 전자 상거래와 같은 다양한 애플리케이션에서 사용되어 사용자에게 빠르고 효율적인 응답을 제공할 수 있습니다.웹사이트, 소셜 미디어 플랫폼 및 메시징 앱과 같은 다양한 채널을 통해 액세스할 수 있습니다.


## Amazon Bedrock을 사용하는 챗봇

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)


## 사용 사례
- 1.**챗봇 (기본)** - FM 모델을 사용하는 제로샷 챗봇
- 2.**프롬프트를 사용하는 챗봇** - 템플릿 (Langchain) - 프롬프트 템플릿에 일부 컨텍스트가 제공된 챗봇
- 3.**페르소나가 있는 챗봇** - 정의된 역할을 가진 챗봇. 즉, 커리어 코치와 인간 상호작용
- 4.**컨텍스트 인식 챗봇** - 임베딩을 생성하여 외부 파일을 통해 컨텍스트를 전달합니다.

## Amazon Bedrock으로 챗봇을 구축하기 위한 랭체인 프레임워크
챗봇과 같은 대화형 인터페이스에서는 단기적 수준뿐만 아니라 장기적 수준에서도 이전 상호 작용을 기억하는 것이 매우 중요합니다.

LangChain은 메모리 구성 요소를 두 가지 형태로 제공합니다.먼저 LangChain은 이전 채팅 메시지를 관리하고 조작하기 위한 도우미 유틸리티를 제공합니다.모듈식으로 설계되어 사용 방식에 관계없이 유용하게 사용할 수 있습니다.둘째, LangChain은 이러한 유틸리티를 체인에 통합하는 쉬운 방법을 제공합니다.
이를 통해 다양한 유형의 추상화를 쉽게 정의하고 상호 작용할 수 있으므로 강력한 챗봇을 쉽게 구축할 수 있습니다.

## 컨텍스트를 활용한 챗봇 구축하기 - 핵심 요소

컨텍스트 인식 챗봇을 구축하는 첫 번째 프로세스는 컨텍스트에 대한**임베딩을 생성**하는 것입니다.일반적으로 임베딩 모델을 통해 실행되고 일종의 벡터 저장소에 저장될 임베딩을 생성하는 통합 프로세스가 있습니다.이 예제에서는 이를 위해 Amazon Titan 임베딩 모델을 사용하고 있습니다.

![Embeddings](./images/embeddings_lang.png)

두 번째 프로세스는 사용자 요청 오케스트레이션, 상호 작용, 호출 및 결과 반환입니다.

![Chatbot](./images/chatbot_lang.png)

## 아키텍처 [컨텍스트 인식 챗봇]
![4](./images/context-aware-chatbot.png)


### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.


이 노트북에는 몇 가지 추가 종속성도 필요합니다.

- [FAISS](https://github.com/facebookresearch/faiss), 벡터 임베딩 저장용
- [iPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), 노트북의 대화형 UI 위젯용
- [PyPDF](https://pypi.org/project/pypdf/), PDF 파일 처리용

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [2]:
#%pip install --quiet "faiss-cpu>=1.7,<2" "ipywidgets>=7,<8" langchain==0.0.249 "pypdf>=3.8,<4"

# 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


# 2. 챗봇(기본 - 컨텍스트 없음)

우리는 LangChain의 [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain)을 사용하여 시작합니다. 대화. 또한 메시지 저장을 위해 [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html)를 사용합니다. 메시지 목록으로 기록을 얻을 수도 있습니다(채팅 모델에서 매우 유용합니다).

챗봇은 이전 상호작용을 기억해야 합니다. 대화 기억을 통해 우리는 그렇게 할 수 있습니다. 대화형 메모리를 구현하는 방법에는 여러 가지가 있습니다. LangChain의 맥락에서 이들은 모두 ConversationChain 위에 구축됩니다.

**참고:** 모델 출력은 비결정적입니다.

In [4]:
from utils.chat import chat_utils
from langchain.llms.bedrock import Bedrock
from langchain.chains import ConversationChain

In [5]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512,
        "temperature": 1,
        "top_k": 250,
        "top_p": 0.999,
        "stop_sequences": ["\n\nHuman:"]
    }
)

## Conversation momory
* **[ConversationBufferMemory](https://python.langchain.com/docs/modules/memory/types/buffer)**
    - This memory allows for storing messages and then extracts the messages in a variable.
* **[ConversationBufferWindowMemory](https://python.langchain.com/docs/modules/memory/types/buffer_window)**
    - It keeps a list of the interactions of the conversation over time.
    - It only uses the last K interactions.
    - This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.
* **[ConversationSummaryBufferMemory](https://python.langchain.com/docs/modules/memory/types/summary_buffer)**
    - It maintains a summary of previous messages.
    - It combines the two ideas.
    - It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both. 
    - It uses token length rather than number of interactions to determine when to flush interactions.

In [6]:
memory = chat_utils.get_memory(
    memory_type="ConversationBufferMemory",
    memory_key="history"
)

conversation = ConversationChain(
    llm=llm_text,
    verbose=True,
    memory=memory
)
print(conversation)

memory=ConversationBufferMemory(return_messages=True) verbose=True llm=Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f49f7c2c910>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512, 'temperature': 1, 'top_k': 250, 'top_p': 0.999, 'stop_sequences': ['\n\nHuman:']})


In [7]:
print_ww(conversation.predict(input="안녕하세요?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: 안녕하세요?
AI:

> Finished chain.
 네, 안녕하세요!


### 결과 분석

여기서 무슨 일이 일어나는가? 우리는 "안녕하세요!"라고 말했습니다. 모델은 몇 가지 대화를 나눴습니다. 이는 Langchain ConversationChain에서 사용하는 기본 프롬프트가 Claude에 맞게 잘 설계되지 않았기 때문입니다. 
- [효과적인 클로드 프롬프트](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)는 `\n\nHuman\n\nAassistant:`로 끝나야 합니다. 이 문제를 해결해 보겠습니다.

Claude의 프롬프트 작성 방법에 대해 자세히 알아보려면 [Anthropic 문서](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)를 확인하세요.

# 3. 프롬프트 템플릿(Langchain)을 이용한 챗봇

LangChain은 프롬프트를 쉽게 구성하고 작업할 수 있도록 여러 클래스와 기능을 제공합니다. [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) 클래스를 사용하여 f-string 템플릿에서 프롬프트를 구성하겠습니다.

**[TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.**

In [8]:
from langchain import PromptTemplate

In [9]:
# turn verbose to true to see the full logs and documents
conversation = ConversationChain(
    llm=llm_text,
    verbose=False,
    memory=memory
)

claude_prompt = PromptTemplate(
        input_variables=["history", 'input'],
        template="""
        \n\nHuman: Here's a friendly conversation between a user and an AI.
        The AI is talkative and provides lots of contextualized details.
        If it doesn't know, it will honestly say that it doesn't know the answer to the question.

        Current conversation:
        {history}

        \n\nUser: {input}

        \n\nAssistant:
        """
)

print("claude_prompt: \n", claude_prompt)

claude_prompt: 
 input_variables=['history', 'input'] template="\n        \n\nHuman: Here's a friendly conversation between a user and an AI.\n        The AI is talkative and provides lots of contextualized details.\n        If it doesn't know, it will honestly say that it doesn't know the answer to the question.\n\n        Current conversation:\n        {history}\n\n        \n\nUser: {input}\n\n        \n\nAssistant:\n        "


In [10]:
conversation.prompt = claude_prompt
print_ww(conversation.predict(input="안녕하세요?"))

 네, 안녕하세요! 반갑습니다.


#### (1) 새로운 질문

모델이 초기 메시지로 응답했습니다. 몇 가지 질문을 해보겠습니다.

In [11]:
print_ww(conversation.predict(input="새로운 정원을 시작하는 방법에 대한 몇 가지 팁을 알려주세요."))

 네, 새로운 정원을 시작하는 몇 가지 팁을 알려드리겠습니다:

- 먼저 정원에서 재배하고 싶은 작물을 결정하세요. 계절별로 재배 가능한 채소, 과일, 꽃 등을 생각해보세요.

- 그 다음은 토질을 테스트하는 것이 좋습니다. 토질의 pH, 영양분 구성 등을 알면 어떤 작물에 적합한지 알 수 있어요.

- 정원의 위치도 중요합니다. 햇빛, 바람, 물 공급 등을 고려하세요.

- 모종이나 씨앗을 구매하기 전에 작물을 심을 준비된 화단이 필요합니다.

- 수분공급이 중요하므로 물뿌리개 시스템을 갖추거나 주수가 용이한 곳을 정하세요.

- 여름 작물이 끝나고 가을 작물을 위해 토양을 깊이 갈아엎는 것이 좋습니다.

- 해충 방지와 병충해 관리를 위해 안전한 농약을 준비하는 것도 잊지 마세요.

정성들여 가꾸다보면 정원에서 새싹들이 돋고 꽃과 열매가 맺히는 것을 볼 수 있을 것입니다. 즐거운 정원 가꾸기 되세요!


#### Check momory

In [12]:
print_ww(memory.load_memory_variables({}))

{'history': [HumanMessage(content='안녕하세요?'), AIMessage(content=' 네, 안녕하세요!'),
HumanMessage(content='안녕하세요?'), AIMessage(content=' 네, 안녕하세요! 반갑습니다.'), HumanMessage(content='새로운
정원을 시작하는 방법에 대한 몇 가지 팁을 알려주세요.'), AIMessage(content=' 네, 새로운 정원을 시작하는 몇 가지 팁을 알려드리겠습니다:\n\n- 먼저 정원에서
재배하고 싶은 작물을 결정하세요. 계절별로 재배 가능한 채소, 과일, 꽃 등을 생각해보세요. \n\n- 그 다음은 토질을 테스트하는 것이 좋습니다. 토질의 pH, 영양분 구성 등을
알면 어떤 작물에 적합한지 알 수 있어요.\n\n- 정원의 위치도 중요합니다. 햇빛, 바람, 물 공급 등을 고려하세요. \n\n- 모종이나 씨앗을 구매하기 전에 작물을 심을 준비된
화단이 필요합니다. \n\n- 수분공급이 중요하므로 물뿌리개 시스템을 갖추거나 주수가 용이한 곳을 정하세요.\n\n- 여름 작물이 끝나고 가을 작물을 위해 토양을 깊이 갈아엎는
것이 좋습니다. \n\n- 해충 방지와 병충해 관리를 위해 안전한 농약을 준비하는 것도 잊지 마세요.\n\n정성들여 가꾸다보면 정원에서 새싹들이 돋고 꽃과 열매가 맺히는 것을 볼
수 있을 것입니다. 즐거운 정원 가꾸기 되세요!')]}


#### (2) 질문을 토대로 작성

모델이 이전 대화를 이해할 수 있는지 확인하기 위해 정원이라는 단어를 언급하지 않고 질문해 보겠습니다.

In [13]:
print_ww(conversation.predict(input="좋아요. 토마토에도 어울릴까요?"))

 네, 토마토도 새로운 정원에 잘 자랄 수 있습니다.

토마토를 심을 때 유의할 점은 다음과 같습니다:

- 토마토는 일교차가 크고 햇볕이 잘 드는 곳을 좋아합니다.

- 토질은 수분 배출이 잘 되면서도 비옥한 사질양토가 적당합니다.

- 토마토 모종을 구입하여 이른 봄에 정원에 심는 것이 좋습니다.

- 지주를 세워 높이 50-100cm 가량 만들어 주는 것이 수확량을 늘릴 수 있습니다.

- 수확기에는 주 1회 정도 관수하되 뿌리 부분이 젖지 않도록 주의합니다.

- 익혀주기, 가지치기, 병충해 방제 등 관리가 필요합니다.

정성껏 가꾸다보면 토마토에서 풍성한 수확을 얻을 수 있을 것입니다. 정원 가꾸기 잘 되세요!


In [14]:
print_ww (memory.load_memory_variables({}))

{'history': [HumanMessage(content='안녕하세요?'), AIMessage(content=' 네, 안녕하세요!'),
HumanMessage(content='안녕하세요?'), AIMessage(content=' 네, 안녕하세요! 반갑습니다.'), HumanMessage(content='새로운
정원을 시작하는 방법에 대한 몇 가지 팁을 알려주세요.'), AIMessage(content=' 네, 새로운 정원을 시작하는 몇 가지 팁을 알려드리겠습니다:\n\n- 먼저 정원에서
재배하고 싶은 작물을 결정하세요. 계절별로 재배 가능한 채소, 과일, 꽃 등을 생각해보세요. \n\n- 그 다음은 토질을 테스트하는 것이 좋습니다. 토질의 pH, 영양분 구성 등을
알면 어떤 작물에 적합한지 알 수 있어요.\n\n- 정원의 위치도 중요합니다. 햇빛, 바람, 물 공급 등을 고려하세요. \n\n- 모종이나 씨앗을 구매하기 전에 작물을 심을 준비된
화단이 필요합니다. \n\n- 수분공급이 중요하므로 물뿌리개 시스템을 갖추거나 주수가 용이한 곳을 정하세요.\n\n- 여름 작물이 끝나고 가을 작물을 위해 토양을 깊이 갈아엎는
것이 좋습니다. \n\n- 해충 방지와 병충해 관리를 위해 안전한 농약을 준비하는 것도 잊지 마세요.\n\n정성들여 가꾸다보면 정원에서 새싹들이 돋고 꽃과 열매가 맺히는 것을 볼
수 있을 것입니다. 즐거운 정원 가꾸기 되세요!'), HumanMessage(content='좋아요. 토마토에도 어울릴까요?'), AIMessage(content=' 네, 토마토도
새로운 정원에 잘 자랄 수 있습니다. \n\n토마토를 심을 때 유의할 점은 다음과 같습니다:\n\n- 토마토는 일교차가 크고 햇볕이 잘 드는 곳을 좋아합니다. \n\n- 토질은
수분 배출이 잘 되면서도 비옥한 사질양토가 적당합니다. \n\n- 토마토 모종을 구입하여 이른 봄에 정원에 심는 것이 좋습니다.\n\n- 지주를 세워 높이 50-100cm 가량
만들어 주는 것이 수확량을 늘릴 수 있습니

#### (3) 대화를 마치며

In [15]:
print_ww(conversation.predict(input="그게 다야, 고마워!"))

 네, 처음 하는 정원 가꾸기에 대한 제안이 도움이 되었길 바랍니다. 토마토 재배에 대한 추가 정보도 제공해드렸구요. 새로운 정원 가꾸기 잘 되시길 바랍니다! 문의사항이나 궁금한
점이 더 있다면 언제든지 말씀해 주세요. 즐거운 정원 가꾸기 되세요!


# 4. ipywidgets를 사용한 대화형 세션

다음 유틸리티 클래스를 사용하면 Claude와 보다 자연스러운 방식으로 상호 작용할 수 있습니다. 입력창에 질문을 적고 클로드의 답변을 받습니다. 그러면 대화를 계속할 수 있습니다.

In [17]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

#### (1) 채팅을 시작해 보겠습니다. 다음 질문을 테스트할 수도 있습니다.

1. 농담 하나 해줘
2. 또 다른 농담을 들려주세요
3. 첫 번째 농담은 무엇이었나요?
4. 첫 번째 농담과 같은 주제로 또 다른 농담을 할 수 있나요?

위의 4가지를 순서대로 아래에 입력하시고, "Send" 버튼을 눌러 보세요.

In [ ]:
chat = ChatUX(conversation)
chat.start_chat()

# 5.페르소나를 활용한 챗봇

AI 비서가 커리어 코치 역할을 하게 됩니다. 
- 역할극 대화에서는 채팅을 시작하기 전에 사용자 메시지를 설정해야 합니다. ConversationBufferMemory는 대화 상자를 미리 채우는 데 사용됩니다.

### (1) ConversationChain 생성 필요한 메모리 초기화, Bedrock Claude 설정

In [18]:
# llm
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 1000,
        "temperature": 1,
        "top_k": 250,
        "top_p": 0.999,
        "stop_sequences": ["\n\nHuman:"]
    }
)

# memory
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory = chat_utils.get_memory(
    memory_type="ConversationBufferMemory",
    memory_key="history"
)

memory.chat_memory.add_user_message("당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다")
memory.chat_memory.add_ai_message("나는 직업 코치이며 직업에 대한 조언을 제공합니다")

# conversation chain
conversation = ConversationChain(
    llm=llm_text,
    verbose=True,
    memory=memory
)
print(conversation)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""
\n\nHuman: Here's a friendly conversation between a user and an AI.
The AI is talkative and provides lots of contextualized details.
If it doesn't know, it will honestly say that it doesn't know the answer to the question.

Current conversation:
{history}

User: {input}

\n\nAssistant:
"""
)


print("claude_prompt: \n", claude_prompt)
conversation.prompt = claude_prompt

memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다'), AIMessage(content='나는 직업 코치이며 직업에 대한 조언을 제공합니다')]), return_messages=True) verbose=True llm=Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f49f7c2c910>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 1000, 'temperature': 1, 'top_k': 250, 'top_p': 0.999, 'stop_sequences': ['\n\nHuman:']})
claude_prompt: 
 input_variables=['history', 'input'] template="\n\n\nHuman: Here's a friendly conversation between a user and an AI.\nThe AI is talkative and provides lots of contextualized details.\nIf it doesn't know, it will honestly say that it doesn't know the answer to the question.\n\nCurrent conversation:\n{history}\n\nUser: {input}\n\n\n\nAssistant:\n"


### (2) 인공지능 관련 직업 질문

In [19]:
# print_ww(conversation.predict(input="“AI의 직업 종류는 무엇입니까?"))
print_ww(conversation.predict(input="“인공지능에 관련된 직업은 어떤 것이 있습니까?"))



> Entering new ConversationChain chain...
Prompt after formatting:



Human: Here's a friendly conversation between a user and an AI.
The AI is talkative and provides lots of contextualized details.
If it doesn't know, it will honestly say that it doesn't know the answer to the question.

Current conversation:
[HumanMessage(content='당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다'), AIMessage(content='나는 직업 코치이며 직업에 대한 조언을 제공합니다')]

User: “인공지능에 관련된 직업은 어떤 것이 있습니까?



Assistant:


> Finished chain.
 네, 인공지능과 관련된 다양한 직업이 있습니다.

- 인공지능 연구원 - 인공지능 알고리즘과 기술을 연구하고 개발합니다.

- 인공지능 엔지니어 - 인공지능 시스템을 설계, 구축, 테스트 하고 유지관리합니다.

- 머신러닝 엔지니어 - 머신러닝 모델을 개발하고 학습 데이터를 준비하며 모델을 배포 및 모니터링합니다.

- 데이터 사이언티스트 - 대규모 데이터를 분석하고 인사이트를 도출하며 머신러닝 문제를 정의합니다.

- 로봇공학자 - 인공지능 기술을 활용한 로봇을 설계 및 개발합니다.

- 자연어 처리 엔지니어 - 자연어 인식, 이해, 생성 기술을 개발합니다.

- 컴퓨터 비전 엔지니어 - 이미지 및 동영상 인식 기술을 개발합니다.

이외에도 인공지능은 많은 산업 분야에서 활용되고 있어 관련 직업은 계속 늘어나고 있습니다. 관심 있는 분야를 찾아 전문성을 키우시면 좋을 것 같습니다.


### (3) 인공지능 관련 직업데 대한 세부 질문

In [20]:
print_ww(conversation.predict(input="이 직업들은 실제로 무엇을 하는가요? 재미있나요?"))



> Entering new ConversationChain chain...
Prompt after formatting:



Human: Here's a friendly conversation between a user and an AI.
The AI is talkative and provides lots of contextualized details.
If it doesn't know, it will honestly say that it doesn't know the answer to the question.

Current conversation:
[HumanMessage(content='당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다'), AIMessage(content='나는 직업 코치이며 직업에 대한 조언을 제공합니다'), HumanMessage(content='“인공지능에 관련된 직업은 어떤 것이 있습니까?'), AIMessage(content=' 네, 인공지능과 관련된 다양한 직업이 있습니다. \n\n- 인공지능 연구원 - 인공지능 알고리즘과 기술을 연구하고 개발합니다. \n\n- 인공지능 엔지니어 - 인공지능 시스템을 설계, 구축, 테스트 하고 유지관리합니다.\n\n- 머신러닝 엔지니어 - 머신러닝 모델을 개발하고 학습 데이터를 준비하며 모델을 배포 및 모니터링합니다.\n\n- 데이터 사이언티스트 - 대규모 데이터를 분석하고 인사이트를 도출하며 머신러닝 문제를 정의합니다. \n\n- 로봇공학자 - 인공지능 기술을 활용한 로봇을 설계 및 개발합니다.\n\n- 자연어 처리 엔지니어 - 자연어 인식, 이해, 생성 기술을 개발합니다. \n\n- 컴퓨터 비전 엔지니어 - 이미지 및 동영상 인식 기술을 개발합니다.\n\n이외에도 인공지능은 많은 산업 분야에서 활용되고 있어 관련 직업은 계속 늘어나고 있습니다. 관심 있는 분야를 찾아 전문성을 키우시면 좋을 것 같습니다.')]

User: 이 직업들은 실제

#### Check memory

In [21]:
pprint(memory.load_memory_variables({}))

{'history': [HumanMessage(content='당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다'),
             AIMessage(content='나는 직업 코치이며 직업에 대한 조언을 제공합니다'),
             HumanMessage(content='“인공지능에 관련된 직업은 어떤 것이 있습니까?'),
             AIMessage(content=' 네, 인공지능과 관련된 다양한 직업이 있습니다. \n\n- 인공지능 연구원 - 인공지능 알고리즘과 기술을 연구하고 개발합니다. \n\n- 인공지능 엔지니어 - 인공지능 시스템을 설계, 구축, 테스트 하고 유지관리합니다.\n\n- 머신러닝 엔지니어 - 머신러닝 모델을 개발하고 학습 데이터를 준비하며 모델을 배포 및 모니터링합니다.\n\n- 데이터 사이언티스트 - 대규모 데이터를 분석하고 인사이트를 도출하며 머신러닝 문제를 정의합니다. \n\n- 로봇공학자 - 인공지능 기술을 활용한 로봇을 설계 및 개발합니다.\n\n- 자연어 처리 엔지니어 - 자연어 인식, 이해, 생성 기술을 개발합니다. \n\n- 컴퓨터 비전 엔지니어 - 이미지 및 동영상 인식 기술을 개발합니다.\n\n이외에도 인공지능은 많은 산업 분야에서 활용되고 있어 관련 직업은 계속 늘어나고 있습니다. 관심 있는 분야를 찾아 전문성을 키우시면 좋을 것 같습니다.'),
             HumanMessage(content='이 직업들은 실제로 무엇을 하는가요? 재미있나요?'),
             AIMessage(content=' 네, 인공지능 관련 직업들이 실제로 하는 일과 재미있는 점에 대해 더 자세히 설명드리겠습니다.\n\n인공지능 연구원은 새로운 알고리즘과 기술을 연구하고 개발하는 일을 합니다. 수학과 컴퓨터 공학 지식을 바탕으로 창의적인 문제해결을 시도하며, 실험을 통해 아이디어를 검증하는 과정이 재미있습니다. \n\n인공지능 엔지니어는 

# 6. 맥락을 가진 챗봇

## 상황에 맞는 챗봇
이 사용 사례에서는 Chatbot에게 이전에 본 적이 없는 외부 코퍼스의 질문에 답변하도록 요청합니다. 이를 위해 RAG(Retrieval Augmented Generation)라는 패턴을 적용합니다. 아이디어는 말뭉치를 덩어리로 인덱싱한 다음 덩어리와 질문 사이의 의미론적 유사성을 사용하여 말뭉치의 어느 섹션이 답변을 제공하는 데 관련될 수 있는지 찾는 것입니다. 마지막으로 가장 관련성이 높은 청크가 집계되어 기록을 제공하는 것과 유사하게 ConversationChain에 컨텍스트로 전달됩니다.

**Titan Embeddings Model**을 사용하여 벡터를 생성하겠습니다. 그런 다음 이 벡터는 메모리 내 벡터 데이터 저장소를 제공하는 Amazon OpenSearch에 저장됩니다. 챗봇이 질문을 받으면 OpenSearch에 질문을 쿼리하고 의미상 가장 가까운 텍스트를 검색합니다. 이것이 우리의 대답이 될 것입니다.

## 6.1 2022년 아마존 주주 서한 문서로 구현 (PDF 문서)

### Amazon Titan Embedding 모델 사용 
- model_id="amazon.titan-embed-text-v1"
- 이 모델은 최대 8,192 Token 입력이 가능합니다.

In [22]:
from langchain.embeddings import BedrockEmbeddings

In [23]:
bedrock_embeddings = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id=bedrock_info.get_model_id(
        model_name="Titan-Embeddings-G1"
    )
)

### PyPDFDirectoryLoader 를 통한 PDF 파일 로딩
- chunk_size 는 임베딩 모델의 최대 입력 토큰 (8,192) 수를 고려하여 정해야 합니다.

In [27]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, SpacyTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

In [38]:
loader = PyPDFDirectoryLoader("./rag_data_kr_pdf/")

* RecursiveCharacterTextSplitter: charater based splitter

In [39]:
documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1024,
    chunk_overlap=256,
    # separators = ['\n'],
    # separators = ['\n','\n\n']
)
docs = text_splitter.split_documents(documents)

In [40]:
docs[:2]

[Document(page_content='CEO로서 두 번째 연례 주주 서한을 작성하기 위해 자리에 앉았을 때 저는 Amazon의 앞날에 대해 낙관적이고 활력을 얻었습니다. 2022년은 최근 기억에 있어 어려운 거시경제적 해 중 하나이고 우리 자체의 운영상의 어려움에도 불구하고 우리는 여전히 수요를 늘릴 방법을 찾았습니다(팬데믹 전반기에 경험한 전례 없는 성장에 더해). 우리는 장단기적으로 고객 경험을 의미 있게 개선하기 위해 대규모 사업을 혁신했습니다. 그리고 고객, 주주 및 직원을 위해 Amazon의 미래를 바꿀 수 있다고 믿는 장기 투자를 계속 유지하면서 투자 결정과 앞으로 나아갈 방법에 중요한 조정을 했습니다.  작년에 이례적인 수의 동시 도전 과제가 있었지만 현실은 유능하고 자금이 충분한 경쟁자가 많은 크고 역동적인 글로벌 시장 부문에서 운영하는 경우(아마존이 모든 비즈니스를 운영하는 조건) 조건이 오랫동안 정체되어 있는 경우는 드뭅니다.  제가 Amazon에 근무한 25년 동안 끊임없는 변화가 있었고 그 중 대부분은 우리가 스스로 시작했습니다. 내가 1997년에 Amazon에 입사했을 때 우리는 1996년에 1,500만 달러의 매출을 올렸고, 도서 전용 소매업체였으며, 제3자 시장이 없었고, 미국 내 주소로만 배송되었습니다. 오늘날 Amazon은 단위 판매의 60%를 차지하고 전 세계 거의 모든 국가의 고객에게 도달하는 활기찬 타사 판매자 에코시스템을 통해 상상할 수 있는 거의 모든 물리적 및 디지털 소매 품목을 판매합니다. 마찬가지로 클라우드에서 일련의 기술 인프라 서비스를 중심으로 비즈니스를 구축하는 것은 AWS를 추구하기 시작한 2003년에는 분명하지 않았으며 2006년 첫 서비스를 출시했을 때도 마찬가지였습니다. 거의 모든 책을 60초 안에 손끝에서 볼 수 있습니다 2007년 Kindle을 출시했을 때 가벼운 디지털 리더에 저장하고 검색할 수 있다는 것은 아직 "사물"이 아니었고 Alexa(2014년 출시)와 같이 액세스하는 데 사용할 수 

* SpacyTextSplitter: sentence based splitter

In [49]:
documents = loader.load()
text_splitter = SpacyTextSplitter(
    chunk_size=1024,
    chunk_overlap=128,
    separator="", # seperator to be inserted between sentences 
    pipeline="ko_core_news_md"
)
docs = text_splitter.split_documents(documents)

In [50]:
docs[:2]

[Document(page_content='CEO로서 두 번째 연례 주주 서한을 작성하기 위해 자리에 앉았을 때 저는 Amazon의 앞날에 대해 낙관적이고 활력을 얻었습니다.\n\n2022년은 최근 기억에 있어 어려운 거시경제적 해 중 하나이고 우리 자체의 운영상의 어려움에도 불구하고 우리는 여전히 수요를 늘릴 방법을 찾았습니다(팬데믹 전반기에 경험한 전례 없는 성장에 더해).\n\n우리는 장단기적으로 고객 경험을 의미 있게 개선하기 위해 대규모 사업을 혁신했습니다.\n\n그리고 고객, 주주 및 직원을 위해 Amazon의 미래를 바꿀 수 있다고 믿는 장기 투자를 계속 유지하면서 투자 결정과 앞으로 나아갈 방법에 중요한 조정을 했습니다.  \n\n작년에 이례적인 수의 동시 도전 과제가 있었지만 현실은 유능하고 자금이 충분한 경쟁자가 많은 크고 역동적인 글로벌 시장 부문에서 운영하는 경우(아마존이 모든 비즈니스를 운영하는 조건) 조건이 오랫동안 정체되어 있는 경우는 드뭅니다.  \n\n제가 Amazon에 근무한 25년 동안 끊임없는 변화가 있었고 그 중 대부분은 우리가 스스로 시작했습니다.\n\n내가 1997년에 Amazon에 입사했을 때 우리는 1996년에 1,500만 달러의 매출을 올렸고, 도서 전용 소매업체였으며, 제3자 시장이 없었고, 미국 내 주소로만 배송되었습니다.\n\n오늘날 Amazon은 단위 판매의 60%를 차지하고 전 세계 거의 모든 국가의 고객에게 도달하는 활기찬 타사 판매자 에코시스템을 통해 상상할 수 있는 거의 모든 물리적 및 디지털 소매 품목을 판매합니다.\n\n마찬가지로 클라우드에서 일련의 기술 인프라 서비스를 중심으로 비즈니스를 구축하는 것은 AWS를 추구하기 시작한 2003년에는 분명하지 않았으며 2006년 첫 서비스를 출시했을 때도 마찬가지였습니다.\n\n거의 모든 책을 60초 안에 손끝에서 볼 수 있습니다 2007년 Kindle을 출시했을 때 가벼운 디지털 리더에 저장하고 검색할 수 있다는 것은 아직 "사물"이 아니었고 Alexa

### OpenSearch Client 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
### 아래 셀에 다음의 정보가 입력이 되어야 합니다.
```
opensearch_domain_endpoint = "<Type Domain Endpoint>"
http_auth = (rag_user_name, rag_user_password) # Master username, Master password

```

In [ ]:
from utils.opensearch import opensearch_utils

In [ ]:
aws_region = 'us-east-1'

os.environ["OpenSearch_UserName"] = "<Type UserName>"
os.environ["OpenSearch_UserPassword"] = "<Type Password>"

rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]

opensearch_domain_endpoint = "<Type your domain endpoint>"

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

### OpenSearch 벡터 Indexer 생성
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

#### 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [ ]:
index_name = "genai-demo-chatbot-index-v1"
index_exists = opensearch_utils.check_if_index_exists(os_client, index_name)

if index_exists:
    opensearch_utils.delete_index(os_client, index_name)
else:
    print("Index does not exist")

#### 인덱스 생성

In [ ]:
from langchain.vectorstores import OpenSearchVectorSearch

In [ ]:
%%time
# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
vector_db = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=docs,
    embedding=bedrock_embeddings,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=10000,
    timeout=60,
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

#### 인덱스 확인

In [ ]:
index_info = os_client.indices.get(index=index_name)
pprint(index_info)

### 은전한잎 형태소 분석기 (seunjeon_tokenizer) 사용하기
- 영어권의 문자들과 다르게 한글, 일본어, 중국어 등은 단순한 공백만으로는 좋은 검색 결과를 얻기 힘듭니다.
- 출시하고라는 단어가 들어간 문서를 출시하고라는 정확히 같은 단어만으로 검색할 수 있다면 답답하겠죠?
- 출시하고라는 단어를 출시, 출시하고 등 다양하게 검색하기 위해서는 형태소 분석기가 필요합니다.
- https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/stemming#

#### 인덱싱 수정하기 (형태소 분석시 사용 enablement)

In [ ]:
new_index_name = f'{index_name}-with-tokenizer'
new_index_name

In [ ]:
# Setting for "은전한잎" Tokenizer (변경 없음)
index_info[index_name]["settings"]["analysis"] = {
    "tokenizer": {
        "seunjeon": {
            "type": "seunjeon_tokenizer"
        }
    },
    "analyzer": {
        "my_analyzer": {
            "type": "custom",
            "tokenizer": "seunjeon"
        }
    }
}

# Setting for Columns to be adapted by Tokenizer (tokenizer가 적용될 컬럼에 맞춰서 수정)
index_info[index_name]["mappings"]["properties"]["text"]["analyzer"] = "my_analyzer"
index_info[index_name]["mappings"]["properties"]["text"]["search_analyzer"] = "my_analyzer"

# Setting for vector index column (변경 없음)
index_info[index_name]["settings"]["index"] = {
    "number_of_shards": "5",
    "knn.algo_param": {"ef_search": "512"},
    "knn": "true",
    "number_of_replicas": "2"
}
del index_info[index_name]["aliases"]
new_index_info = index_info[index_name]

In [ ]:
pprint(new_index_info)

#### 형태소 분석기용 인덱서 생성

In [ ]:
index_exists = opensearch_utils.check_if_index_exists(os_client, new_index_name)
if index_exists:
    opensearch_utils.delete_index(os_client, new_index_name)
else:
    print("Index does not exist")

In [ ]:
opensearch_utils.create_index(
    os_client,
    index_name=new_index_name,
    index_body=new_index_info
)

#### Re-indexing

In [ ]:
_reindex = {
    "source": {"index": index_name},
    "dest": {"index": new_index_name}
}
print("_reindex: \n", _reindex)

In [ ]:
os_client.reindex(_reindex)

#### 키워드 검색 체크

In [ ]:
query = "아마존"
query = opensearch_utils.get_query(
    query=query
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

#### 형태소 분석 결과 확인
"아마존" 확인 <BR>
#### [중요]:  doc_id: 위의 문서 인덱스 정보 확인 후 수정

In [ ]:
doc_id = "f30b455b-c1cd-421a-ada2-2a80efd2bd2d"

In [ ]:
os_client.termvectors(index=new_index_name, id=doc_id, fields='text')

### 문서 및 임베딩 확인

In [ ]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

In [ ]:
print("docs[0].page_content: \n", docs[0].page_content)

In [ ]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

### Hybrid search

LangChain에서 제공하는 Wrapper 클래스를 사용하여 벡터 데이터베이스 저장소를 쿼리하고 관련 문서를 반환할 수 있습니다. 뒤에서는 RetrievalQA 체인만 실행됩니다.

In [ ]:
from utils.rag import OpenSearchHybridSearchRetriever

In [ ]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    vector_db=vector_db,
    index_name=new_index_name,
    k=3,
    fusion_algorithm="simple_weighted", # ["RRF", "simple_weighted"]
    ensemble_weights=[.2, .8], # [lexical, semantic]
    verbose=False
    
)

In [ ]:
query = "아마존은 Generative AI 의 전략이 무엇인가요?"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print(f'question: {query}')
print(f'response: {search_hybrid_result}', len(search_hybrid_result))

하이브리드(lexical + semantic) 검색이 어떻게 작동하는지 살펴보겠습니다.
1. 먼저 쿼리에 대한 임베딩 벡터를 계산하고
2. 그런 다음 이 벡터를 사용하여 벡터 스토어에서 유사성 검색을 수행한다.
3. document 풀을 기반으로 쿼리에 대한 full text 검색을 수행한다.
4. 두가지 결과를 바탕으로 re-ranking을 한다.
5. 결과를 반환한다.

In [ ]:
v = bedrock_embeddings.embed_query(query)
print(v[0:10], len(v))
for r in search_hybrid_result:
    print_ww(r.page_content)
    print('----')

### 메모리
모든 챗봇에는 사용 사례에 따라 맞춤화된 다양한 옵션을 갖춘 QA 체인이 필요합니다. 그러나 챗봇에서는 모델이 답변을 제공하기 위해 이를 고려할 수 있도록 항상 대화 기록을 보관해야 합니다. 이 예에서는 대화 기록을 유지하기 위해 ConversationBufferMemory와 함께 LangChain의 [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db)을 사용합니다.

출처: https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db

뒤에서 무슨 일이 일어나고 있는지 모두 보려면 'verbose'를 'True'로 설정하세요.

In [ ]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

In [ ]:
print (CONDENSE_QUESTION_PROMPT.template)

### ConversationRetrievalChain에 사용되는 매개변수
* **retriever**: 우리는 `OpenSearch`를 기반으로 customization한 `OpenSearchHybridSearchRetriever`를 사용했습니다. 이것은 `OpenSearch`를 이용하여 주어진 `query`에 대한 `lexical`, `sematic` search를 기반으로 최종 후보 context 를 선택해 줍니다.

* **메모리**: 이력을 저장하는 메모리 체인

* **condense_question_prompt**: 사용자의 질문이 주어지면 이전 대화와 해당 질문을 사용하여 독립형 질문을 구성합니다.

* **chain_type**: 채팅 기록이 길고 상황에 맞지 않는 경우 이 매개변수를 사용하고 옵션은 `stuff`, `refine`, `map_reduce`, `map-rerank`입니다.

질문이 컨텍스트 범위를 벗어나면 모델은 답을 모른다고 대답합니다.

**참고**: 체인이 어떻게 작동하는지 궁금하다면 `verbose=True` 줄의 주석 처리를 해제하세요.

In [ ]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [ ]:
# memory
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory_chain = chat_utils.get_memory(
    memory_type="ConversationBufferMemory",
    memory_key="chat_history",
    return_messages=True
)

llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 1000,
        "temperature": 1,
        "top_k": 250,
        "top_p": 0.999,
        "stop_sequences": ["\n\nHuman:"]
    }
)

In [ ]:
# turn verbose to true to see the full logs and documents
from langchain.schema import BaseMessage
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nUser: ", "ai": "\n\nAI: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nUser: " + dialogue_turn[0]
            ai = "\n\nAI: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# 이전 대화 내용을 기반으로, 신규 질문을 재구성 하는 prompt
condense_prompt_claude = PromptTemplate.from_template("""
\n\nHuman:
Given the following conversation and a follow up question,
rephrase the follow up question to be a standalone question, in its original language.

Chat History: {chat_history}
Follow Up Input: {question}

\n\nAssistant: Question:"""
)

# recreate the Claude LLM with more tokens to sample - this provide longer responses but introduces some latency
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=0,
    filter=[],
    verbose=False
)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm_text, 
    retriever=opensearch_hybrid_retriever,
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    verbose=True,
    condense_question_prompt=condense_prompt_claude,
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
\n\nHuman:

{context}

<q></q> XML 태그 내의 질문에 답하려면 최대 3개의 문장을 사용하세요.

<q>{question}</q>

답변에 XML 태그를 사용하지 마세요.
답변이 context에 없으면 "죄송합니다. 문맥에서 답을 찾을 수 없어서 모르겠습니다."라고 말합니다.

\n\nAssistant:""")

채팅을 해보죠. 아래와 같은 질문을 해보세요. 
1. 누가 쓴 편지인가요?
2. 몇 년도에 썼나요?
3. 현재 누가 아마존을 이끌고 있나요?
4. Generative AI 의 전략이 무엇인가요?

In [ ]:
from utils.chat import ChatUX

In [ ]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

## Delete OpenSearch Index

In [ ]:
print(index_name, new_index_name)

In [ ]:
for name in [index_name, new_index_name]:
    
    index_exists = opensearch_utils.check_if_index_exists(os_client, name)
    if index_exists:
        opensearch_utils.delete_index(os_client, name)
    else:
        print("Index does not exist")

# 7.Next Action

자신만의 챗봇 시스템을 만들고 챗봇과 대화한 내역을 캡쳐해서 올려주세요. 웹 페이지 정보를 가져오기 위해서 'WebBaseLoader' 를 사용하는 것도 가능합니다.

### 요약
- 이 데모에서는 Claude LLM을 사용하여 다음 패턴으로 대화형 인터페이스를 만들었습니다.

1. 챗봇(기본 - 맥락 없음)

2. 프롬프트 템플릿(Langchain)을 이용한 챗봇

3. 페르소나를 갖춘 챗봇
4. 맥락을 갖춘 챗봇